<a href="https://colab.research.google.com/github/sophielouie/beer-recommendation-system/blob/main/GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy import spatial
import statistics as stats
import math
from plotly.subplots import make_subplots
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split
from networkx.algorithms.dag import topological_sort
import networkx as nx
import plotly.graph_objects as go
import pickle

#import Node2Vec and Hybrid Notebooks
import node2vec_module
from node2vec_module import node2vec, clean_beer_reviews
from hybridmodel import hybrid_model, clean_beer_reviews
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Dropdown
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Beer Recommendation
### Welcome to our Nobel Prize winning beer rec algorithms!
### We have created two recommendation systems - a Hybrid model and a Node2Vec model
#### We use a hybrid recommendation system that generates recommendations by first creating a list of the 50 most similar beers to the target beer using beer reviews (content-based) and then utilizes an SVD matrix to predict the beers that the user would rate the highest (collaborative filtering).
#### For our Node2Vec recommedation system, we establish links between beers based on how many attributes they share to form a graph of beer nodes. Node2Vec transforms the graph into vector representations and we use cosine similarity to find beers that re close in proximity, but do not already share an edge. We are working under the assumption that nodes close in distance should have an edge connecting them because Node2Vec preserves the structure of the input graph.
### Directions:
#### 

In [3]:
BEER_DF = clean_beer_reviews()

In [4]:
# make a fucntion that returns a list of beers
def make_beer_list():
    return BEER_DF["Beer Name"].unique()


# list of user, beer combination
def make_beer_user_list(user):
    return BEER_DF.loc[BEER_DF["review_profilename"] == user]["Beer Name"]

# list of users to choose from
def make_user_list():
    return BEER_DF['review_profilename'].unique()

In [5]:
# MDOEL TYPE DROPDOWN TO USER/BEER OPTIONS
def model_dropdown_eventhandler(change):
    determine(model_dropdown.value)

option_list = ["Hybrid", "Node2Vec"]

user = make_user_list()[:100]
beer = make_beer_list()

def determine(x):
    third_dropdown.layout.visibility = "hidden"
    sec_dropdown.index = None
    if x == "Hybrid":
        sec_dropdown.options = user
    else:
        sec_dropdown.options = beer
        
# HYBRID MODEL USER CHOISE TO CORRESPONDING BEER OPTIONS
def beer_dropdown_eventhandler(change):
    determine_beer(sec_dropdown.value)

def determine_beer(x):
    third_dropdown.index = None
    beer_user = make_beer_user_list(x)
    if len(beer_user) > 0:
        third_dropdown.options = beer_user
        third_dropdown.layout.visibility = "visible"
    else: 
        third_dropdown.layout.visibility = "hidden"

model_dropdown = Dropdown(description="Select a model", options=option_list,
            style = {'description_width': 'initial'}, value=None)
#This will be users for when hybrid is selected, beers for when Node2Vec is selected
sec_dropdown = Dropdown(description="Choose one",
            style = {'description_width': 'initial'})
model_dropdown.observe(model_dropdown_eventhandler, names='value')
#This will only appear when Hybrid is selected and contains beer list
third_dropdown = Dropdown(description="Choose one", layout = {"visibility" : "hidden"},
            style = {'description_width': 'initial'})
sec_dropdown.observe(beer_dropdown_eventhandler, names='value')
num_recs = widgets.IntSlider(
            value=5,
            min=0,
            max=10,
            step=1,
            description='Number of recommendations:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
            layout=widgets.Layout(width='50%', height='40px'),
            style = {'description_width': 'initial'}
        )

display(model_dropdown, sec_dropdown, third_dropdown, num_recs)

Dropdown(description='Select a model', options=('Hybrid', 'Node2Vec'), style=DescriptionStyle(description_widt…

Dropdown(description='Choose one', options=(), style=DescriptionStyle(description_width='initial'), value=None…

Dropdown(description='Choose one', layout=Layout(visibility='hidden'), options=(), style=DescriptionStyle(desc…

IntSlider(value=5, continuous_update=False, description='Number of recommendations:', layout=Layout(height='40…

In [6]:
btn = widgets.Button(description='Start')
out = widgets.Output(layout={'border': '1px solid black'})

display(btn, out)

def btn_eventhandler(obj):
    out.clear_output()
    
    if model_dropdown.value == 'Hybrid':
        if sec_dropdown.value and third_dropdown.value and num_recs:
            with out:
                print('running')
                result = hybrid_model(sec_dropdown.value, third_dropdown.value, num_recs.value)
                print(result['Beer Name'])
    
    elif model_dropdown.value == 'Node2Vec':
        if sec_dropdown.value and num_recs:
            with out:
                print('running')
                print(node2vec(sec_dropdown.value, num_recs.value))
    
btn.on_click(btn_eventhandler)

Button(description='Start', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))